In [3]:
# importing the module
import pandas as pd
import numpy as np
from os.path import join as opj
 
DATA_DIR = "../data/raw_swis"    
sites_excel = "Sites.xlsx"
activities_excel = "SiteActivities.xlsx"
operators_excel = "SiteOperators.xlsx"
owners_excel = "SiteOwners.xlsx"
waste_excel = "SiteWaste.xlsx"

# reading the files
sites = pd.read_excel(opj(DATA_DIR, sites_excel))    
#sites = pd.read_excel("../data/swis/Sites.xlsx")
activities = pd.read_excel(opj(DATA_DIR, activities_excel)) 
# operators = pd.read_excel(opj(DATA_DIR, operators_excel))
# owners = pd.read_excel(opj(DATA_DIR, owners_excel))
waste = pd.read_excel(opj(DATA_DIR, waste_excel))


In [4]:
sites
# activities


,SWIS Number,Site Name,Activity,Activity Is Archived,OperationalStatus,RegulatoryStatus,Category,Activity Classification,WDRNo,WDR Landfill Class,...,SWRCB Region,Local Government,Street Address,City,State,ZIP Code,Reporting Agency Legal Name,Reporting Agency Department,Enforcing Agency Legal Name,Enforcing Agency Department
0,01-AA-0006,Davis Street Sanitary Landfill,Solid Waste Disposal Site,No,Closed,Permitted,Disposal,Solid Waste Disposal Site,NaN,NaN,...,San Francisco Bay,San Leandro,W End Of Davis St.,San Leandro,CA,94577,County of Alameda,Department of Environmental Health,County of Alameda,Department of Environmental Health
1,01-AC-0029,Cty of Berk Solid Waste Mgmt Center & TS,Large Volume Transfer/Processing Facility,No,Active,Permitted,Transfer/Processing,Solid Waste Facility,NaN,NaN,...,San Francisco Bay,Berkeley,1201 2nd St,Berkeley,CA,94710,City of Berkeley,--,Department of Resources Recycling and Recovery,"Waste Permitting, Compliance and Mitigation Di..."
2,01-CR-0005,Old West Winton Landfill,Solid Waste Disposal Site,No,Closed,Pre-regulations,Disposal,Solid Waste Disposal Site,NaN,NaN,...,San Francisco Bay,Hayward,S Side Of W End W Winton Rd,Hayward,CA,94545,County of Alameda,Department of Environmental Health,County of Alameda,Department of Environmental Health
3,01-CR-0031,Crow Canyon SWDS,Solid Waste Disposal Site,No,Closed,To Be Determined,Disposal,Solid Waste Disposal Site,NaN,NaN,...,San Francisco Bay,Hayward,8795 Norris Canyon Rd,Hayward,CA,94552,County of Alameda,Department of Environmental Health,County of Alameda,Department of Environmental Health
4,03-CR-0005,Old Sutter Creek Dump,Solid Waste Disposal Site,No,Closed,Unpermitted,Disposal,Solid Waste Disposal Site,NaN,NaN,...,Central Valley,Sutter Creek,"13680 Shake Ridge Rd,1000` Fr Pine Gulch",Sutter Creek,CA,95685,County of Amador,Environmental Health Department,County of Amador,Environmental Health Department
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3070,33-AA-0374,MDE Compost Site 1,Green Material Composting Facility,No,Active,Notification,Composting,Solid Waste Facility,NaN,NaN,...,Colorado River,Riverside County (Unincorporated),Ave 68 & Harrison,Thermal,CA,92274,County of Riverside,Department of Environmental Health,County of Riverside,Department of Environmental Health
3071,37-CR-0131,2112 Crest Drive IDS,Solid Waste Disposal Site,No,Inactive,Unpermitted,Disposal,Solid Waste Disposal Site,NaN,NaN,...,San Diego,El Cajon,2112 Crest Drive,El Cajon,CA,92021,County of San Diego,Department of Environmental Health,County of San Diego,Department of Environmental Health
3072,37-CR-0132,Old HWY 80 IDS,Solid Waste Disposal Site,No,Inactive,Unpermitted,Disposal,Solid Waste Disposal Site,NaN,NaN,...,Colorado River,San Diego County (Unincorporated),42748 Old Highway 80,Jacumba,CA,91934,County of San Diego,Department of Environmental Health,County of San Diego,Department of Environmental Health
3073,33-AA-0378,Verdant Plastics Recycling Facility,Large Volume Transfer/Processing Facility,No,Planned,Proposed,Transfer/Processing,Solid Waste Facility,NaN,NaN,...,Santa Ana,Riverside County (Unincorporated),4388 Serrano Drive,Mira Loma,CA,91752,County of Riverside,Department of Environmental Health,County of Riverside,Department of Environmental Health


In [48]:
# merging the files
    
all_df_list = [sites, activities, operators, owners, waste]

dataframe = sites
for file in [activities, operators, owners, waste]:
    dataframe = pd.merge(dataframe, file, on = "SWIS Number", how = "outer")




In [49]:
dataframe.

,SWIS Number,Site Name_x,Point of Contact_x,Site Operational Status_x,Site Regulatory Status_x,Is Archived_x,IsClosedIllegalAbandoned_x,IsSiteInertDebrisEngineeredFill_x,IsFinancialAssurancesResponsible_x,Absorbed On_x,...,Site Regulatory Status,Site Type,Latitude,Longitude,Point of Contact,Activity Is Archived_y,Activity Operational Status,Activity Regulatory Status,Activity Category,Activity Classification_y
0,01-AA-0006,Davis Street Sanitary Landfill,Sabra Ambrose,Closed,Permitted,No,Yes,No,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-AA-0014,Therm-Tec Of California,Alyssa Williams,Planned,Proposed,Yes,No,No,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-AA-0273,Marciel Ranch Landspreading,Sabra Ambrose,Closed,Exempt,Yes,No,No,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-AC-0029,Cty of Berk Solid Waste Mgmt Center & TS,Lily Tieu,Active,Permitted,No,Yes,No,No,NaN,...,Permitted,Non-Disposal Only,37.88034,-122.30579,Lily Tieu,No,Active,Permitted,Transfer/Processing,Solid Waste Facility
4,01-AC-0029,Cty of Berk Solid Waste Mgmt Center & TS,Lily Tieu,Active,Permitted,No,Yes,No,No,NaN,...,Permitted,Non-Disposal Only,37.88034,-122.30579,Lily Tieu,No,Active,Permitted,Transfer/Processing,Solid Waste Facility
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11323,37-CR-0132,Old HWY 80 IDS,Abel Martinez-Centeno,Inactive,Unpermitted,No,No,No,No,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11324,33-AA-0378,Verdant Plastics Recycling Facility,Theodore Tasiopoulos,Planned,Proposed,No,No,No,No,NaN,...,Proposed,Non-Disposal Only,34.01206,-117.50951,Theodore Tasiopoulos,No,Planned,Proposed,Transfer/Processing,Solid Waste Facility
11325,19-AR-1275,Easy Waste Management,Benjamin Escotto,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,34.25207,-118.39885,Benjamin Escotto,No,Active,Notification,Transfer/Processing,Solid Waste Operation
11326,19-AR-1275,Easy Waste Management,Benjamin Escotto,Active,Notification,No,No,No,No,NaN,...,Notification,Non-Disposal Only,34.25207,-118.39885,Benjamin Escotto,No,Active,Notification,Transfer/Processing,Solid Waste Operation
